# send and receive torch model

In [5]:
import torch
import torchvision # model을 불러오기 위해 import
import torch.onnx

def make_vgg16_model(fname):
    # 1. 실제 사용하는 custom model을 불러와서 저장 가능
    model = torchvision.models.vgg16(pretrained=False)

    # 2. model의 파라미터를 OrderedDict 형태로 저장
    params = model.state_dict()

    # 3. 동적 그래프 형태의 pytorch model을 위하여 data를 model로 흘려주기 위한 더미 데이터 주입
    dummy_data = torch.empty(1, 3, 224, 224, dtype = torch.float32)

    # 4. onnx 파일을 export 함. 함수에는 차례대로 model, data, 저장할 파일명 순서대로 입력
    torch.onnx.export(model, dummy_data, fname)


import pika

fname = "send.onnx"
make_vgg16_model(fname)
f=open(fname, "rb")
img=f.read()

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.queue_declare(queue='torch_model')

channel.basic_publish(exchange='',routing_key='torch_model',body=img)
print("[+] Sent image")
connection.close()



[+] Sent image


In [4]:
import pika, sys, os
import time


def main():

    connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
    channel = connection.channel()

    channel.queue_declare(queue='torch_model')

    def callback(ch, method, properties, body):
        #print(" [x] Received %r" % body)
        print("[+] ok, received !")

        try:
            fname = received.onnx
            f=open(fname, "wb")
            f.write(body)
            f.close()
        
            print("[+] ok, saved !")
        except:
            print("[-] failed, file saving")

    channel.basic_consume(queue='torch_model', on_message_callback=callback, auto_ack=True)

    print(' [*] Waiting for messages. To exit press CTRL+C')
    channel.start_consuming()

if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        print('Interrupted')

        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)



 [*] Waiting for messages. To exit press CTRL+C
Interrupted
